In [2]:
import os
import csv
import copy
import math
import time
import random
import numpy as np
import pandas as pd
import seaborn as sns
import networkx as nx
from scipy.stats import mode
from numba import njit, typed
import matplotlib.pyplot as plt
from scipy.stats import norm, gaussian_kde

## Used Functions

In [1]:
def create_noisy_copies(graph, K, f, verbose=False):
    """
    Function that creates K copies with an percentage error of 'f' of a graph.
    
    Inputs:
        graph: nx.DiGraph()
            Original graph from which K copies will be created (this will be the true underlying graph).
            
        K: int
            Number of copies to create from the original graph.
    
        f: float (from 0 to 1, both inclusive)
            The parameter 'f' can be thought of as a measure of the "error" or "noise" in the
            observations. A higher 'f' means more noise, resulting in higher probabilities of
            altering the graph (adding or removing edges).
            
        verbose: bool, optional (default: verbose=False)
            Set to True if information about the copies is desired. Change it to False otherwise.
    
    Returns:
        obs: list of nx.DiGraph()
            List of the K noisy copies that have been created.
    
    
    """
    
    obs = []
    nodes = list(graph.nodes())
    edges = list(graph.edges())
    num_edges = len(edges)
    non_edges = [(u,v) for u in nodes for v in nodes if u!=v and not graph.has_edge(u,v)]
    
    # Calculate the number of edges to swap
    swapping_edges = int(f*num_edges)
    
    for i in range(K):
        graph_copy = graph.copy()
            
        # Randomly take an edge and non-edge in the graph
        random_edges = random.sample(edges, k=swapping_edges)
        random_non_edges = random.sample(non_edges+random_edges, k=swapping_edges) # Select non-edges from all positions where there's not an edge, including the places where there used to be an edge.

        # Swap them.
        graph_copy.remove_edges_from(random_edges)
        graph_copy.add_edges_from(random_non_edges)              
            
        # And append the modified graph to the list
        obs.append(graph_copy)
        
        if verbose:
            print(f"Observation {i+1}: {swapping_edges} edges have been swapped.")
    
    return obs

In [4]:
@njit
def energy_H(params):
    """
    Mathematical function that expresses the energy of an alignment with a blueprint given all of its parameters.
    
    Inputs:
        params: array
            Array that contains some parameters regarding edge and non-edge matching between nodes in the blueprint
            and observations, edges and non-edges in the blueprint, number of observations and hyperparameters for
            the assumed prior distributions of the probabilities p and q (in this order).
            
            Expected array:
                params = [O_00, O_01, O_10, O_11, n_0, n_1, n_obs, alpha_p, beta_p, alpha_q, beta_q]
            
    Returns:
        H: float
            Value of the energy H of the corresponding alignment.
            
    """
    
    O_00, O_01, O_10, O_11, n_0, n_1, K, alpha_p, beta_p, alpha_q, beta_q = params
    
    H = -(math.lgamma(O_11+beta_q) + math.lgamma(O_10+alpha_q) + math.lgamma(O_00+beta_p) + math.lgamma(O_01+alpha_p) - math.lgamma(K*n_1 + alpha_q + beta_q) - math.lgamma(K*n_0 + alpha_p + beta_p))
    return H

In [5]:
def energy_config_H(observations,alignment,L):
    """
    Computes the posterior probability given a set of observations, an alignment and a blueprint L.

    Inputs:
        observations: list of nx.DiGraph()
            A list of observed graphs (one for each observation). Each graph represents a noisy observation of the 
            underlying structure of the blueprint graph L.

        alignment: np.matrix
            A matrix where each row corresponds to a particular observation, and each column represents the alignment 
            of nodes across observations.
            
            Example:
                [[0,2,1]
                 [0,1,2]
                 [2,1,0]
                 [1,0,2]]
            This would be an alignment with 4 observations containing three nodes each. In this case,
            the nodes 0, 0, 2 and 1 from observations 0, 1, 2 and 3 (respectively) are mapped to
            the same node in the blueprint; and so on for the rest of the nodes (columns).
            
        L: nx.DiGraph()
            The blueprint graph representing the underlying true structure of the noisy observations.

    Returns:
        H: float
            The energy of the alignment based on the given observations and blueprint graph.
    
    """


    list_of_nodes = list(observations[0].nodes())
    n_nodes = len(list_of_nodes)
    n_obs = len(observations)
        
    O_00,O_01,O_10,O_11 = 0,0,0,0

    all_positions = [(a,b) for a in range(n_nodes) for b in range(n_nodes) if a!=b]
    
    for i, j in all_positions:
        node_i_in_L, node_j_in_L = list_of_nodes[i],list_of_nodes[j]
        for k in range(n_obs):
            node_i_in_A, node_j_in_A = alignment[k,i],alignment[k,j]
            if L.has_edge(node_i_in_L,node_j_in_L):
                if observations[k].has_edge(node_i_in_A,node_j_in_A):
                    O_11 += 1
                else:
                    O_10 += 1
            else:
                if observations[k].has_edge(node_i_in_A,node_j_in_A):
                    O_01 += 1
                else:
                    O_00 += 1   
    
    n_1 = L.number_of_edges()
    n_0 = n_nodes**2-n_nodes-n_1 # Number of non-edges in L.
            
    alpha_p,beta_p,alpha_q,beta_q = 0.2,10,0.5,5
    params = np.array([O_00,O_01,O_10,O_11,n_0,n_1,n_obs,alpha_p,beta_p,alpha_q,beta_q], dtype=np.float64)
    H = energy_H(params=params)

    return H

In [6]:
def initialize_alignment_by_degree(observations, seeds=None, group_labels=None):
    """
    Initializes an alignment matrix based on node degree, grouping nodes into different categories: seeds, 
    group-labeled nodes, and nodes with no additional information (aligned by degree). The function returns 
    both the alignment matrix and the column positions for each node category. The aligned seed nodes will
    be placed in the first columns of the alignment matrix, then each group label, and finally the nodes
    with no additional information.
    
    Inputs:
        observations: list of nx.DiGraph()
            A list of directed graphs representing the observed networks. These are assumed to be
            noisy versions of an underlying true structure.
   
            
        seeds: dict, optional (default: seeds=None)
            A dictionary where each key corresponds to the observation, and the associated value is a list of seed 
            nodes from that observation that are pre-aligned with the blueprint graph (i.e. that its true alignment
            is known beforehand). These seeds are used to fix node mappings across observations.

            Example:
                seeds = {0: [0,1], 1: [6,4], 2: [2,2], 3: [5,3], 4: [3,5]}

                In this example, nodes 0, 6, 2, 5 and 3 from observation 0, 1, 2, 3 and 4 (respectively) are mapped
                to the same nodes in the blueprint (same for the nodes 1, 4, 2, 3 and 5).
                These seeds cannot be swapped during the MCMC process.
    
        group_labels: list of dict, optional (default: group_labels=None)
            A list where each element is a dictionary for a specific group label. The dictionary keys are observation 
            indices, and the values are lists of node IDs that are grouped together. Nodes with the same group label 
            across observations can only be mapped to each other.

            Example:
                group_labels = [
                    {0: [2,3], 1: [2,3], 2: [2,3], 3: [2,3], 4: [2,3]}, 
                    {0: [4,5,6], 1: [4,5,6], 2: [4,5,6], 3: [4,5,6], 4: [4,5,6]}
                ]
            
            In this example, nodes 2 and 3 from all five observations share the same group label, so they can only 
            be mapped to each other and not to any other nodes. Similarly, nodes 4, 5, and 6 form another group.
                
    Returns:
        alignment: numpy.ndarray
            A matrix where each row corresponds to a particular observation, and each column represents the alignment 
            of nodes across observations. Nodes are grouped first by seeds, then by group-labeled nodes, and finally
            by nodes with no additional information (which are aligned based on their degree).
            
        positions: dict
            A dictionary that tracks the column positions for each node category (seeds, group-labeled nodes, and nodes 
            without additional information). The keys are the category names, and the values are the column ranges 
            corresponding to those categories in the 'alignment' matrix.

            Example:
                positions = {
                    "seeds": range(0, 2), 
                    "group_label_0": range(2, 4), 
                    "group_label_1": range(4, 7), 
                    "no info": range(7, 10)
                }

            This indicates that the first two columns correspond to seeds, columns 2 and 3 to group label 0,
            columns 4 to 6 to group label 1, and columns 7 to 9 to nodes without additional information.
                
    """
    
    list_of_nodes = list(observations[0].nodes())
    
    n_rows = len(observations)
    n_cols = len(list_of_nodes)
    
    # Initialize empty lists for matrices and positions
    matrices = []
    positions = {}
    

    # SEEDS
    if seeds is not None:
        num_seeds = len(seeds[0])  # Assuming same number of seeds across observations
        seeds_matrix = np.empty((n_rows, num_seeds), dtype=np.int64)
        
        for observ, nodes_seeds in seeds.items():
            seeds_matrix[observ, :num_seeds] = nodes_seeds  # Each row 'i' gets its corresponding seed alignments
        
        matrices.append(seeds_matrix)
        positions["seeds"] = range(0, num_seeds) # Column positions for seeds


    # GROUP LABELS
    if group_labels is not None:
        for i,group_label in enumerate(group_labels):
            num_nodes_label = len(group_label[0])
            group_matrix = np.empty((n_rows, num_nodes_label), dtype=np.int64)

            col_start = sum(len(m[0]) for m in matrices)  # Start column index for this group
            for observ, nodes_label in group_label.items():
                group_matrix[observ, :num_nodes_label] = nodes_label  # Fill with group nodes

            matrices.append(group_matrix)
            positions[f"group_label_{i}"] = range(col_start, col_start + num_nodes_label)  # Column positions for this group

        
    # NODES WITHOUT INFORMATION
    no_info = {}
    for k in range(n_rows):
        nodes_no_info = [node for node in observations[k].nodes() if 
                        (seeds is None or node not in seeds[k]) and 
                        (group_labels is None or all(node not in group_labels[l][k] for l in range(len(group_labels))))]

        sorted_nodes_degree = sorted(nodes_no_info, key=lambda x: observations[k].degree(x), reverse=True)
        no_info[k] = sorted_nodes_degree

    # Determine the maximum length for the no_info_matrix
    num_no_info = len(no_info[0]) if no_info else 0
    no_info_matrix = np.empty((n_rows, num_no_info), dtype=np.int64)

    for observ, nodes_no_info in no_info.items():
        no_info_matrix[observ, :num_no_info] = nodes_no_info

    if num_no_info > 0:
        matrices.append(no_info_matrix)
        col_start = sum(len(m[0]) for m in matrices[:-1])  # Start index for no_info_matrix.
        positions["no info"] = range(col_start, col_start + num_no_info)  # Column positions for no info.

    # Concatenate all matrices into one alignment matrix.
    alignment = np.concatenate(matrices, axis=1)
    
    return alignment, positions

In [7]:
def most_common_mapping(alignment,get_percentage=False):
    """
    Updates the last row of the alignment matrix with the most common node 
    mapping for each node across all observations. If there is a tie, 
    the entry is set to -1.

    Inputs:
        alignment: np.matrix
            An alignment matrix of shape (n_obs, n_nodes).
            
        get_percentage: bool, optional (default: get_percentage=False)
            Sets if the percentage of correctly aligned nodes is calculated or not.

    Returns:
        most_common_mapping: array
            An array that corresponds to the most common mapping of each node or -1 in case of ties.
            
            Example:
                [1,0,2,-1]
                
            This shows that the most common mapping among observation for the first node of the blueprint
            has been the node 1, for the second node, the node 0, then node 2 and finally, for the last
            node of the blueprint, there are two nodes that have been equally aligned to it, therefore
            it's a tie and -1 is shown.
            
    """

    n_obs = alignment.shape[0]
    n_nodes = alignment.shape[1]
    
    correctly_aligned_nodes = 0
    most_common_mapping = []
    
    for n in range(n_nodes):
        counter = {i: 0 for i in range(n_nodes)}
        for o in range(n_obs):
            node = alignment[o,n]
            counter[node] += 1
            
        # Find the node (or nodes) with the maximum count
        max_count = max(counter.values())        
        most_common_nodes = [node for node, count in counter.items() if count == max_count]
        
        if len(most_common_nodes) == 1:
            # Include the most common mapping for that blueprint node in the new row.
            most_common_mapping.append(most_common_nodes[0])
            
            if get_percentage:
                # iF there's a single node representing the majority of aligned nodes among observations, then these are considered as correctly aligned nodes.
                correctly_aligned_nodes += max_count
            
        else:
            # Set the label of that node to NaN
            most_common_mapping.append(-1)
            
            # If there's more than one node being the majority, then all of them are considered to be wrongly aligned.
    
    if get_percentage:
        p = correctly_aligned_nodes*100/(n_obs*n_nodes)
#         print(f"Percentage of correctly aligned nodes: {round(p,2)}%")
        return most_common_mapping,p
    
    return most_common_mapping

In [8]:
@njit
def percentage_well_aligned_nodes(alignment):
    """
    Calculates the percentage of correctly aligned nodes across all observations.
    A node is considered correctly aligned if it has a single most common mapping across all observations.
    
    Inputs:
        alignment: np.matrix
            An alignment matrix of shape (n_obs, n_nodes).
            
    Returns:
        percentage: float
            The percentage of correctly aligned nodes across all nodes and observations.
    """
    n_obs = alignment.shape[0]
    n_nodes = alignment.shape[1]
    
    correctly_aligned_nodes = 0

    # Loop through each node to calculate the most common mapping and check for correctness
    for n in range(n_nodes):
        # Create a counter (list) for the occurrences of each node mapping
        counter = np.zeros(n_nodes, dtype=np.int32)
        
        for o in range(n_obs):
            node = alignment[o,n]
            counter[node] += 1
        
        # Find the maximum count
        max_count = np.max(counter)
        # Check how many nodes have the same count as the maximum
        most_common_count = np.sum(counter == max_count)
        
        if most_common_count == 1:
            # If there's a single most common node, it's correctly aligned
            correctly_aligned_nodes += max_count
    
    # Calculate the percentage of correctly aligned nodes
    percentage = correctly_aligned_nodes * 100 / (n_obs * n_nodes)
    
    return percentage

In [9]:
def create_L(E,n_obs):
    """
    Creates a directed graph L based on the adjacency matrix E and a threshold on n_obs.
    
    Inputs:
        E: np.matrix
            An adjacency matrix where each entry E[i,j] indicates the number of observations that have the edge (i,j).
        n_obs, int
            The number of observations; used to determine the threshold for edge inclusion.
    
    Returns:
        L: nx.DiGraph
            A directed graph where edges (i, j) are included if E[i,j] >= 0.5*n_obs.
            
    """
    
    L = nx.DiGraph()
    n_nodes = E.shape[0]
    L.add_nodes_from(np.arange(n_nodes))
    
    all_possible_edges = [(a,b) for a in range(n_nodes) for b in range(n_nodes) if a!=b]
    edges = []

    for i,j in all_possible_edges:
        if E[i,j] > 0.5*n_obs:
            edges.append((i,j))
    
    L.add_edges_from(edges)
    return L

In [10]:
@njit
def swap_and_update_vars(alignment,adj_matrices_repl,E_old,O_00_old,O_01_old,O_10_old,O_11_old,n_1_old,list_of_nodes,random_obs_idx,x,y):
    """
    """
    
    # Total number of observations and nodes.
    n_obs,n_nodes = alignment.shape[0],alignment.shape[1]
    
#     # Select a random observation and two random nodes from it:

#     # Obs:
#     random_obs_idx = np.random.randint(0,n_obs)

#     # Nodes:
#     x = np.random.randint(0, n_nodes)
#     y = np.random.choice(np.delete(np.arange(n_nodes), x))  # y = random.choice([i for i in range(n_nodes) if i != random_idx_node_1])

#     node_1 = new_alignment[random_obs_idx,x]
#     node_2 = new_alignment[random_obs_idx,y]
        
#     print(f"Obs: {random_obs_idx}. Nodes: {node_1}, {node_2}. Positions: {x}, {y}")

    # Take the adjacent matrix of the randomly chosen DiGraph from that replica
    adj_matrix = adj_matrices_repl[random_obs_idx].copy()

    E_permut = E_old.copy() # Using .copy() is important, as later on 'E_old' will be used. Without it, any modification of 'E_permut' would also affect 'E_old', and the previous data would be lost!
    E_permut -= adj_matrix

    # Swap the rows in the randomly chosen positions.
    temp_row = adj_matrix[x, :].copy()
    adj_matrix[x, :] = adj_matrix[y, :]
    adj_matrix[y, :] = temp_row

    # Same with the columns.
    temp_col = adj_matrix[:, x].copy()
    adj_matrix[:, x] = adj_matrix[:, y]
    adj_matrix[:, y] = temp_col

    E_permut += adj_matrix
    E_new = E_permut

    
    # The new variables are initialized to the old values, and they will be updated according to the proposed swap.
    O_00_new,O_01_new,O_10_new,O_11_new,n_1_new = O_00_old,O_01_old,O_10_old,O_11_old,n_1_old

    positions_to_check = [(a,b) for a in range(n_nodes) for b in range(n_nodes) if a!=b and (a==x or a==y or b==x or b==y)]

    
    # Iterate over the positions of the edges to check
    for a,b in positions_to_check:
        node_a,node_b = list_of_nodes[a],list_of_nodes[b] # alignments[j][n_obs,a],alignments[j][n_obs,b] 
#                 print(f"Observations with the edge ({node_a},{node_b}): {E_new[a,b]}")

        # The new L has the edge:
        if E_new[a,b] > 0.5*n_obs:

            # The previous L also had the edge:
            if E_old[node_a,node_b] > 0.5*n_obs:
                O_10_new += E_old[a,b] - E_new[a,b] # number of old edges - number of new edges (as new apparisons of an edge (i,k) in the observations now contributes negatively to O_10, which is the total number of matches between an edge (i,k) being in L but not in all observations).
                O_11_new += E_new[a,b] - E_old[a,b] # the opposite! number of new edges - number of old edges (this will tell us the number of new edges (i,k) that exist in the observations that also exist in L).
                # print(f"O_00={O_00_new}, O_01={O_01_new}, O_10={O_10_new}, O_11={O_11_new}\n")

            # The previous L didn't have the edge:
            else:
                O_00_new -= n_obs - E_old[a,b] # the observations that previously didn't have the edge (node_i,node_j) (same as L)
                O_01_new -= E_old[a,b] # the observations that previously had the edge, in contrast to (the previous) L
                O_10_new += n_obs - E_new[a,b] # the observations that now don't have the edge (and now L does contain it)
                O_11_new += E_new[a,b] # the observations containing the edge (same as L!)
                n_1_new += 1 # The number of edges in the blueprint L increases by 1.
                # print(f"O_00={O_00_new}, O_01={O_01_new}, O_10={O_10_new}, O_11={O_11_new}\n")

        # The new L doesn't have the edge:
        else:

            # The previous L had the edge:
            if E_old[a,b] > 0.5*n_obs:
                O_00_new += n_obs - E_new[a,b] # the observations that now don't have the edge (same as L!) previously didn't have the edge (node_i,node_j) (same as L)
                O_01_new += E_new[a,b] # the observations containing the edge (and now L doesn't contain it)
                O_10_new -= n_obs - E_old[a,b] # the observations that didn't have the edge (and L previously had it)
                O_11_new -= E_old[a,b] # the observations that previously had the edge (same as the previous L)
                n_1_new -= 1 # The number of edges in the blueprint L decreases by 1.
                # print(f"O_00={O_00_new}, O_01={O_01_new}, O_10={O_10_new}, O_11={O_11_new}\n")

            # The previous L neither had the edge:
            else:
                O_00_new += E_old[a,b] - E_new[a,b] # number of old edges - number of new edges (as new apparisons of an edge (i,k) in the observations now contributes negatively to O_00, which is the total number of matches between an edge (i,k) not being in L and also not being in observations).
                O_01_new += E_new[a,b] - E_old[a,b] # the opposite! number of new edges - number of old edges (this will tell us the number of new edges (i,k) that exist in the observations that also exist in L).
                # print(f"O_00={O_00_new}, O_01={O_01_new}, O_10={O_10_new}, O_11={O_11_new}\n")
                

    n_0_new = n_nodes**2-n_nodes-n_1_new # Number of non-edges in L, and n_1_new is the number of edges in (the new) L.

    return O_00_new,O_01_new,O_10_new,O_11_new,n_0_new,n_1_new,E_new,adj_matrix

In [11]:
@njit
def MCMC(max_mcmc_steps,n_replicas,alignments,adj_matrices,E_old,H_old,O_00_old,O_01_old,O_10_old,O_11_old,n_0_old,n_1_old,list_of_nodes,verbose,H_true,stop):
    """
    """
    # Total number of observations and nodes.
    n_obs,n_nodes = alignments[0].shape[0],alignments[0].shape[1]
    
    H_min = 99999999
    align_min = np.empty((n_obs, n_nodes), dtype=np.int64) # DEBUG? 32?
    
    alignable = 100 # Alignability as a percentage, so at the beginning it is assumed that we will be able to find the best alignment.
    same_or_best_align = False # Has the true alignment or a better one been found?
    steps = -1
    
    k = 1
    T = 1.05**np.linspace(0,30,n_replicas)
    beta = 1/(k*T)
    
    info = np.empty((n_replicas, max_mcmc_steps))  # Pre-allocate a NumPy array to save the results for each replica after each MCMC step
    percentage_align = np.empty(max_mcmc_steps, dtype=np.float64)
    O_00_vals = np.empty(max_mcmc_steps, dtype=np.int64)
    O_01_vals = np.empty(max_mcmc_steps, dtype=np.int64)
    O_10_vals = np.empty(max_mcmc_steps, dtype=np.int64)
    O_11_vals = np.empty(max_mcmc_steps, dtype=np.int64)
    
    # CURRENTLY DISABLED.
#     # Here will be stored the alignments that will be saved every 20 MCMC steps.
#     sampled_alignments = []
    
    # MCMC algorithm with parallel tempering starts.
    for i in range(max_mcmc_steps):
        for l in range(n_nodes//2 * n_obs):
            for j in range(n_replicas):
            
    #             print(f"\n\nIteration {i}: replica {j} ----->")
    
                # Select a random observation and pair of nodes.
                random_obs_idx = np.random.randint(0,n_obs)
                x,y = np.random.choice(n_nodes, size=2, replace=False)
                node_1, node_2 = alignments[j][random_obs_idx,x], alignments[j][random_obs_idx,y]
                
                # Propose a new alignment and compute its energy H_new.
                O_00_new, O_01_new, O_10_new, O_11_new, n_0_new, n_1_new, E_new, adj_matrix = \
                swap_and_update_vars(
                    alignment=alignments[j], 
                    adj_matrices_repl=adj_matrices[j], 
                    E_old=E_old[j], 
                    O_00_old=O_00_old[j], 
                    O_01_old=O_01_old[j], 
                    O_10_old=O_10_old[j], 
                    O_11_old=O_11_old[j], 
                    n_1_old=n_1_old[j], 
                    list_of_nodes=list_of_nodes,
                    random_obs_idx=random_obs_idx,
                    x=x,
                    y=y
                )
                                
                # Calculate the new energy.
                alpha_p,beta_p,alpha_q,beta_q = 0.2,10,0.5,5
                params = np.array([O_00_new,O_01_new,O_10_new,O_11_new,n_0_new,n_1_new,n_obs,alpha_p,beta_p,alpha_q,beta_q], dtype=np.float64)
                H_new = energy_H(params=params)

                # Accept it via the Metropolis algorithm.
                p_swapping = min(1,math.exp(beta[j]*(H_old[j]-H_new)))
                rand = random.random() if p_swapping < 1 else 0

                if p_swapping == 1 or p_swapping > rand:
    #                 print("Improvement: nodes are swapped.")
                    # Accept the swapping:

                    # Accept the change: Update the current iteration to be the newest.
                    O_00_old[j],O_01_old[j],O_10_old[j],O_11_old[j],n_0_old[j],n_1_old[j] = O_00_new,O_01_new,O_10_new,O_11_new,n_0_new,n_1_new # DEBUG WARNING!
                    E_old[j],H_old[j] = E_new,H_new
                    # Update the current alignment.
                    alignments[j][random_obs_idx,x],alignments[j][random_obs_idx,y] = node_2,node_1
                    adj_matrices[j][random_obs_idx] = adj_matrix

                # Otherwise, the swap is rejected, and therefore no changes are made.
                
                # Save the minimum energy throughout the algorithm and the alignment that produced it.
                if H_old[j] < H_min:
                    H_min = H_old[j]
                    align_min = alignments[j]
                
                # Check the number of steps done when the true alignment or a better one has been found.
                if not same_or_best_align:
                    if H_old[j]/n_nodes <= H_true+0.0000001: # Some tolerance, just in case Python makes decimal mistakes.
                        steps = i+1
                        same_or_best_align = True
                    

        # After each MCMC Step, propose a collective swap between the alignments in contiguous replicas.
        start, end = i%2, n_replicas-1 # POSSIBLE DEBUG: n_replicas-2 + i%2
        for repl in range(start,end,2):

            p_swapping_replicas = min(1,math.exp(-(beta[repl] - beta[repl+1])*(H_old[repl+1] - H_old[repl])))

            rand = random.random()
            if p_swapping_replicas > rand:
                # Change is accepted (all information about both replicas is swapped).
                O_00_old[repl],O_00_old[repl+1] = O_00_old[repl+1],O_00_old[repl]
                O_01_old[repl],O_01_old[repl+1] = O_01_old[repl+1],O_01_old[repl]
                O_10_old[repl],O_10_old[repl+1] = O_10_old[repl+1],O_10_old[repl]
                O_11_old[repl],O_11_old[repl+1] = O_11_old[repl+1],O_11_old[repl]
                n_0_old[repl],n_0_old[repl+1] = n_0_old[repl+1],n_0_old[repl]
                n_1_old[repl],n_1_old[repl+1] = n_1_old[repl+1],n_1_old[repl]

                E_old[repl],E_old[repl+1] = E_old[repl+1],E_old[repl]
                H_old[repl],H_old[repl+1] = H_old[repl+1],H_old[repl]

                alignments[repl],alignments[repl+1] = alignments[repl+1],alignments[repl]
                adj_matrices[repl],adj_matrices[repl+1] = adj_matrices[repl+1],adj_matrices[repl]

            else:
                # Change isn't accepted.
#                 print(f"Change NOT accepted with p={p_swapping_replicas}")
                pass

        # Save the energy per node of each MCMC step and replica.
        percentage_align[i] = percentage_well_aligned_nodes(alignment=alignments[0])
        O_00_vals[i] = O_00_old[0]
        O_01_vals[i] = O_01_old[0]
        O_10_vals[i] = O_10_old[0]
        O_11_vals[i] = O_11_old[0]
        for j in range(n_replicas):
            info[j,i] = H_old[j]/n_nodes
            if alignable == 100:
                if info[j,i] < H_true:
                    alignable = 0 # When the energy of an alignment is lower than the true energy, then the latter can't be found.

        if verbose:
            progress = (i+1)/max_mcmc_steps*100
            if round(progress,5) % 5 == 0:
                print("Progress --->", round(progress,2),"%")
        
        # ALIGNMENT AT EQUILIBRIUM, CURRENTLY DISABLED.
        # Store alignments each 20 MCMC steps of the last 500 from the T=1 replica (assumed to be at equilibrium).
#         if i > max_mcmc_steps-500:
#             if i%20 == 0:
#                 sampled_alignments.append(alignments[0])

        if same_or_best_align and stop:
            return alignments, info[:, :steps], percentage_align[:steps], O_00_vals[:steps], O_01_vals[:steps], O_10_vals[:steps], O_11_vals[:steps], T, alignable, H_min/n_nodes, align_min, steps
    
    return alignments, info, percentage_align, O_00_vals, O_01_vals, O_10_vals, O_11_vals, T, alignable, H_min/n_nodes, align_min, steps

In [12]:
def mcmc_paraltemp(obs, sigma, stop=True, folder_name=None, iteration=0, perturb=None, max_mcmc_steps=5000, group_labels=None, seeds=None, n_replicas=20, verbose=True):
    """
    Perform an MCMC with parallel tempering to minimise the energy H = -log(p(L,{pi^k},{A^k})),
    i.e. to optimise the posterior distribution p(L,{pi^k}|{A^k}) = exp(-H)/p({A^k}) \prop \Gamma(O_{11}+\beta_q)*\Gamma(O_{10}+\alpha_q)/\Gamma(Kn_1+\alpha_q+\beta_q) * \Gamma(O_{00}+\beta_p)*\Gamma(O_{01}+\alpha_p)/\Gamma(Kn_0+\alpha_p+\beta_p).
    The parallel tempering will involve 15 replicas exploring the alignment space at the same time at different temperatures.
    
    Inputs:
        obs: list of nx.DiGraph()
            List containing all the graph observations.

        max_mcmc_steps: int, optional (default: max_mcmc_steps=5000)
            Number of maximum iterations (MCMC steps) to perform.
        
        L: nx.DiGraph(), optional (default: L=None)
            If known, true underlying graph (blueprint) of the noisy observations.
            This is used for showing the ground true energy in the graph.

        group_labels: list of dict, optional (default: group_labels=None)
            ......

        seeds: dict, optional (default: seeds=None)
            ......
            
        n_replicas: int, optional (default: n_replicas=20)
            ......
            
        verbose: bool, optional (default: verbose=True)
            Set to True to show the progress of the MCMC algorithm. Set to False otherwise.
            
    Returns:
        alignments: list of np.matrix
            List containing the best alignment for each replica in the MCMC algorithm.

        L: list of nx.DiGraph()
            List containing the best blueprint for each replica in the MCMC algorithm.
    
    """
    
    # Time the execution of the algorithm.
    start = time.time()
        
    # Some initial parameters.
    n_obs = len(obs)
    n_nodes = obs[0].number_of_nodes()
    list_of_nodes = np.array(obs[0].nodes())
    
    # Get the ground true energy H_true.
    alignment = np.tile(np.arange(n_nodes), (n_obs,1)).astype(np.int64) # True alignment
    adj_matrix = np.array([nx.adjacency_matrix(ob, nodelist=alignment[k]).toarray() for k, ob in enumerate(obs)], dtype=np.int64)
    E = np.sum(adj_matrix, axis=0)
    H_true = energy_config_H(observations=obs,alignment=alignment,L=create_L(E,n_obs))/n_nodes
            
    # Matrix that represents the alignment between the nodes in the observations. Initially, they will be aligned by degree.
    if perturb is None:
        alignment, _ = initialize_alignment_by_degree(observations=obs, seeds=seeds, group_labels=group_labels)
    else:
        alignment = perturb_alignment(np.tile(np.arange(n_nodes),(n_obs,1)), perturb).astype(np.int64)
        
    # Initial blueprint graph L
    O_00,O_01,O_10,O_11 = np.int64(0),np.int64(0),np.int64(0),np.int64(0)
    E_initial = np.zeros([n_nodes,n_nodes], dtype=np.int64) # E_ij = number of observations containing i-->j as an edge in L.

    # Initialise the adjacency matrices for each observation, and set the matrix of observations as the sum of the adj. matrices.
    adj_matrix = np.array([nx.adjacency_matrix(ob, nodelist=alignment[k]).toarray() for k, ob in enumerate(obs)], dtype=np.int64)
    E_initial = np.sum(adj_matrix, axis=0)


    n_0,n_1 = np.int64(0),np.int64(0) # Number of non-edges and edges of the initial blueprint, respectively.
    all_positions = [(a,b) for a in range(n_nodes) for b in range(n_nodes) if a!=b]
    
    for i,j in all_positions: # We loop through all possible positions of pairs of nodes.
        
        # If more than half the observations contain the edge (i,j):
        if E_initial[i,j] > 0.5*n_obs:
            # The edge is included in the blueprint L.
            n_1 += 1
            O_11 += E_initial[i,j] # Number of observations that have (i,j) as an edge.
            O_10 += n_obs - E_initial[i,j] # Number of obs. not containing that edge (total observations - obs. containing it)
        
        # If the number of observations containing the edge (i,j) is not the majority,
        else:
            n_0 += 1
            O_01 += E_initial[i,j]
            O_00 += n_obs - E_initial[i,j]

        
    # Calculate the initial energy.
    alpha_p,beta_p,alpha_q,beta_q = 0.2,10,0.5,5
    params = np.array([O_00,O_01,O_10,O_11,n_0,n_1,n_obs,alpha_p,beta_p,alpha_q,beta_q], dtype=np.float64)
    H_initial = energy_H(params=params)
    
    # Define some parameters for the MCMC parallel tempering phase.
    O_00_old, O_01_old, O_10_old, O_11_old, n_0_old, n_1_old = [], [], [], [], [], []
    E_old, H_old = [], []
    alignments = []
    
    # This will be a list of lists of matrices, where adj_matrices[4][2] will represent the adj matrix in the 4th replica of the 2nd observation.
    adj_matrices = []
    
    for j in range(n_replicas):
        O_00_old.append(O_00)
        O_01_old.append(O_01)
        O_10_old.append(O_10)
        O_11_old.append(O_11)
        n_0_old.append(n_0)
        n_1_old.append(n_1)
        
        E_old.append(copy.copy(E_initial))
        H_old.append(H_initial)
        
        alignm = alignment.copy()
        alignments.append(alignm)
        
        adj_mat = adj_matrix.copy()
        adj_matrices.append(adj_mat)
     
    O_00_old = typed.List(O_00_old)
    O_01_old = typed.List(O_01_old)
    O_10_old = typed.List(O_10_old)
    O_11_old = typed.List(O_11_old)

    n_0_old = typed.List(n_0_old)
    n_1_old = typed.List(n_1_old)

    E_old = typed.List(E_old)
    H_old = typed.List(H_old)

    alignments = typed.List(alignments)
    adj_matrices = typed.List(adj_matrices)
    
    # sampled_alignments, CURRENTLY DISABLED.
    alignments, info, percentage_each_step, O_00_vals, O_01_vals, O_10_vals, O_11_vals, T, alignable, H_min, min_alignment, steps = \
    MCMC(
        max_mcmc_steps,
        n_replicas,
        alignments,
        adj_matrices,
        E_old,
        H_old,
        O_00_old,
        O_01_old,
        O_10_old,
        O_11_old,
        n_0_old,
        n_1_old,
        list_of_nodes,
        verbose,
        H_true,
        stop
    )
        
    
    # Obtain the percentage of correctly aligned nodes of the last alignment in the T=1 replica.
    last_alignment = alignments[0]
    _, percent_last = most_common_mapping(alignment=last_alignment,get_percentage=True)
    
    # Obtain the percentage of correctly aligned nodes of the alignment that produced the lowest energy.
    _, percent_min = most_common_mapping(alignment=min_alignment,get_percentage=True)
    
    # ALIGNMENT SAMPLED FROM EQUILIBRIUM CURRENTLY DISABLED.
#     # Obtain the percentage of correctly aligned nodes of the alignments sampled in equilibrium.
#     equi_alignment = most_common_alignment(sampled_alignments)
#     _, percent_equi = most_common_mapping(alignment=equi_alignment,get_percentage=True)
    
    end = time.time()
    

    # Create a CSV file where all information will be saved.
    file_name = f"results_{sigma}_{iteration}.csv"
    if folder_name and os.path.exists(folder_name):
        file_path = os.path.join(folder_name, file_name)
    else:
        file_path = file_name  # Save in the current directory

    # Write a CSV file with all the information of the run.
    with open(file_path, mode="w", newline="") as file:
        writer = csv.writer(file)
    
        # Initial parameters before the run.
        file.write("-------------Parameters:-------------\n")
        parameters = [
            ["Sigma", sigma],
            ["MCMC Steps", max_mcmc_steps],
            ["Observations", n_obs],
            ["Replicas", n_replicas]
        ]
        writer.writerows(parameters)
        
        # Some obtained values after the run.
        values = [
            ["Percentage of correctly aligned nodes",percent_last,percent_min], # , percent_equi DISABLED.
            ["Alignability",alignable],
            ["Steps required to reach the true alignment or find a better one",steps],
            ["H_min and H_true",H_min,H_true],
            ["Time",end-start]
        ]
        writer.writerows(values)

        # Energies.
        file.write("-------------Energies:-------------\n")
        # Create DataFrame for all found energies.
        stps = max_mcmc_steps if (steps == -1 or not stop) else steps
        df_energies = pd.DataFrame(info,index=[f"Replica {p+1}" for p in range(n_replicas)],columns=[f"Step {q+1}" for q in range(stps)])
        df_energies.insert(0,"Temperatures",T)
        info_column = [f"Replica {j+1}" for j in range(n_replicas)]
        df_energies.insert(0,"Info replicas",info_column)
        # Write it in the CSV file.
        df_energies.to_csv(file,index=False,header=False)
        
        # Alignments.
        file.write("-------------Alignments:-------------\n")   
        # Last alignment
        file.write("Last alignment\n")
        df_align = pd.DataFrame(last_alignment)
        df_align.to_csv(file,index=False,header=False)
        # Minimum alignment
        file.write("Minimum alignment\n")
        df_align = pd.DataFrame(min_alignment)
        df_align.to_csv(file,index=False,header=False)
        # ALIGNMENT FROM EQUILIBRIUM DISABLED.
#         # Average alignment from equilibrium
#         file.write("Equilibrium alignment\n")
#         df_align = pd.DataFrame(equi_alignment)
#         df_align.to_csv(file,index=False,header=False)

        # Additional info.
        file.write("-------------Additional info:-------------\n")
        writer.writerow(["Percentages of the best replica per step"]+list(percentage_each_step))
        writer.writerow(["O_00"]+list(O_00_vals))
        writer.writerow(["O_01"]+list(O_01_vals))
        writer.writerow(["O_10"]+list(O_10_vals))
        writer.writerow(["O_11"]+list(O_11_vals))

## Plotting Functions

In [13]:
def plot_energies_csv(file_path,folder_name=None):
    percents, O_00_vals, O_01_vals, O_10_vals, O_11_vals = None, None, None, None, None
    with open(file_path, mode='r', newline='') as file:
        reader = csv.reader(file)
        
        T = []
        energies = []

        # Loop through the rows in the CSV file
        for row in reader:
            if row[0] == "Sigma":
                sigma = float(row[1])
                
            elif row[0] == "MCMC Steps":
                mcmc_total_steps = int(row[1])
            
            elif row[0] == "Replicas":
                n_replicas = int(row[1])
                
            elif row[0] == "H_min and H_true":
                H_min,H_true = float(row[1]),float(row[2])
                
            elif row[0].startswith("Replica"):
                T.append(float(row[1]))
                energies.append([float(H) for H in row[2:]])
                
            elif row[0] == "Percentages of the best replica per step":
                percents = [float(x) for x in row[1:]]
                
            elif row[0] == "O_00":
                O_00_vals = [int(x) for x in row[1:]]

            elif row[0] == "O_01":
                O_01_vals = [int(x) for x in row[1:]]

            elif row[0] == "O_10":
                O_10_vals = [int(x) for x in row[1:]]

            elif row[0] == "O_11":
                O_11_vals = [int(x) for x in row[1:]]


        steps = len(energies[0])
        
        # Plot energies.
        plt.figure(figsize=(25,10))
        plt.title(f"Energy per Node over MCMC Steps for Different Temperatures at σ={sigma}",fontsize=25)
        plt.xlabel("MCMC step",fontsize=18)
        plt.ylabel("Energy per node",fontsize=18)

        # Plot the true energy as a reference.
        plt.axhline(y=H_true, color='blue', linestyle='--', label="Ground true energy per node")

        for j in range(n_replicas):

            if T[j] == 1.0:
                plt.plot(range(steps), energies[j], color='black', label=f'T={T[j]}')
            else:
                plt.plot(range(steps), energies[j], color='pink', label=f'T={T[j]}')

        plt.legend(loc='best')
        
        if folder_name:
            image_path = os.path.join(folder_name, "plot" + file_path.split("results")[1].split(".csv")[0] + ".png")
        else:
            image_path = "plot" + file_path.split("results")[1].split(".csv")[0] + ".png"
            
        plt.savefig(image_path)  # Save as a PNG file.
        
        
        # Plot the percentages per each MCMC step.
        if percents is not None:
            plt.figure(figsize=(25,10))
            plt.title(rf"% of Correctly Aligned Nodes of $T=1$ Replica per MCMC step with $\sigma={sigma}$",fontsize=25)
            plt.xlabel("MCMC step",fontsize=18)
            plt.ylabel("% of Correctly Aligned Nodes",fontsize=18)
            plt.plot(range(steps), percents)
            
            if folder_name:
                image_path = os.path.join(folder_name, "%_align" + file_path.split("results")[1].split(".csv")[0] + ".png")
            else:
                image_path = "%_align" + file_path.split("results")[1].split(".csv")[0] + ".png"
                
            plt.savefig(image_path)  # Save as a PNG file.
        
        
        # Plot the values of O_XY per each MCMC step.
        if O_00_vals is not None:
            plt.figure(figsize=(25,10))
            plt.title(rf"Values of the $O_{{XY}}$ Parameters of the $T=1$ Replica per MCMC step with $\sigma={sigma}$",fontsize=25)
            plt.xlabel("MCMC step",fontsize=18)
            plt.ylabel(r"Values of $O_{XY}$",fontsize=18)
            
            plt.plot(range(steps), O_00_vals, label=r'$O_{00}$')
            plt.plot(range(steps), O_01_vals, label=r'$O_{01}$')
            plt.plot(range(steps), O_10_vals, label=r'$O_{10}$')
            plt.plot(range(steps), O_11_vals, label=r'$O_{11}$')
            
            plt.legend(loc='best',fontsize=18)
            
            if folder_name:
                image_path = os.path.join(folder_name, "O_XY" + file_path.split("results")[1].split(".csv")[0] + ".png")
            else:
                image_path = "O_XY" + file_path.split("results")[1].split(".csv")[0] + ".png"
                
            plt.savefig(image_path)  # Save as a PNG file.
#         plt.show()

In [14]:
def run_mcmc_multiple_params(G, sigma_range, n_obs, folder_name, n_iter, max_mcmc_steps):
    """
    """
    
    if not os.path.exists(folder_name):
        os.mkdir(folder_name)
        print(f"Folder '{folder_name}' created.")
    else:
        print(f"Folder '{folder_name}' already exists.")
    
    for sigma in sigma_range:
        for j in range(n_iter):

            # Generate noisy copies of the graph
            observations = create_noisy_copies(graph=G, K=n_obs, sigma=sigma, verbose=True)

            # Run MCMC parallel tempering
            mcmc_paraltemp(obs=observations,sigma=sigma,folder_name=folder_name,iteration=j,max_mcmc_steps=max_mcmc_steps,verbose=False)
            
            # Plot the graph of its energies
            file_path = os.path.join(folder_name, f"results_{sigma}_{j}.csv")
            plot_energies_csv(file_path,folder_name)

In [9]:
def detectability_plot(folder_name,density=True,detectability=True,avg_steps=True,var_steps=True,avg_H_gaps=True,var_H_gaps=True):
    """
    Plot the percentage of correctly aligned nodes across different sigma values.
    
    Inputs:
        node_range:
            ...
            
        edge_range:
            ...
            
        sigma_range: array 
            Range of sigma values to iterate over.
            
        G: nx.DiGraph()
            Original graph.
            
        n_observations: int 
            Number of noisy copies of the graph.
            
        n_iter: int, optional (default: n_iter=20)
            Number of runs of the MCMC algorithm for each sigma.
            
        max_mcmc_steps: int, optional (default: max_mcmc_steps=5000) 
            Maximum MCMC steps.
        
    Returns:
        ......
        
    """    
    sigmas = []
    last_prob = {}
    min_prob = {}
    align = {}
    steps = {}
    H_gaps = {}
    
    files = [f for f in os.listdir(folder_name) if f.startswith("results")]
    sorted_files = sorted(files, key=lambda x: float(x.split("_")[1]))
    
    for file_name in sorted_files:
        if file_name.startswith('results'):
            file_path = os.path.join(folder_name, file_name)
            processed_flags = {
                "percentage": False,
                "alignability": False,
                "steps": False,
                "H_min_H_true": False
            }
            sigma = float(file_path.split("_")[1])
            if sigma not in sigmas:
                sigmas.append(sigma)
                last_prob[sigma],min_prob[sigma],align[sigma],steps[sigma],H_gaps[sigma] = [],[],[],[],[]

            with open(file_path, mode="r") as file:
                
                # Read each file.
                reader = csv.reader(file)
                for row in reader:
                    if row[0] == "Percentage of correctly aligned nodes":
                        percent_last,percent_min = float(row[1]),float(row[2])
                        min_prob[sigma].append(percent_min)
                        processed_flags["percentage"] = True

                    elif row[0] == "Alignability":
                        alignable = float(row[1])
                        align[sigma].append(alignable)
                        processed_flags["alignability"] = True

                    elif row[0] == "Steps required to reach the true alignment or find a better one":
                        processed_flags["steps"] = True
                        if int(row[1]) != -1:
                            max_steps = int(row[1])
                            steps[sigma].append(max_steps)

                    elif row[0] == "H_min and H_true":
                        H_min,H_true = float(row[1]),float(row[2])
                        H_gaps[sigma].append(H_min-H_true)
                        processed_flags["H_min_H_true"] = True

                    # If all conditions have been processed, break the loop
                    if all(processed_flags.values()):
                        break
                        
    # Calculate over sigma                
    min_prob_plot = [np.mean(prob) for prob in min_prob.values()]
    align_plot = [np.mean(alignable) for alignable in align.values()]
    avg_steps_plot = [np.mean(stp) for stp in steps.values()]
    var_steps_plot = [np.std(stp) for stp in steps.values()]
    avg_H_gaps_plot = [np.mean(H_gps) for H_gps in H_gaps.values()]
    var_H_gaps_plot = [np.std(H_gps) for H_gps in H_gaps.values()]
        

    # Detectability plot
    if detectability:
        plt.title("Percentage of correctly aligned nodes")
        plt.xlabel("Noise level $f$")
        plt.ylabel("Percentage (%)")
        plt.tick_params(axis='both', which='major')
        
        # Alignment Accuracy
        plt.plot(sigmas,min_prob_plot,label='Mean % (μ)')
        
        # Upper and Lower Bounds
        mins_min_prob = [min(prob) for prob in min_prob.values()]
        maxs_min_prob = [max(prob) for prob in min_prob.values()]
        std_min_prob_plot = [np.std(prob) for prob in min_prob.values()]
        upper_bound = [min(maxs_min_prob[i], min_prob_plot[i]+std_min_prob_plot[i]) for i in range(len(sigmas))]
        lower_bound = [max(mins_min_prob[i], min_prob_plot[i]-std_min_prob_plot[i]) for i in range(len(sigmas))]

        plt.plot(sigmas, upper_bound, linestyle='--', label="μ + σ")
        plt.plot(sigmas, lower_bound, linestyle='--', label="μ - σ")
        plt.fill_between(sigmas, lower_bound, upper_bound, color='blue', alpha=0.2)

        # Alignability
        #plt.plot(sigmas,align_plot,color='red',label='Alignability')

        # Additional Info
        plt.legend(loc='best')
        plt.grid(True)
        plt.tight_layout()
        image_path = os.path.join(folder_name, "finalplot.png")
        plt.savefig(image_path)
        plt.show()
    
    if avg_steps:
        # Plot the average number of steps.
        plt.title("Average number of steps to find the true alignment (or a better one)")
        plt.xlabel("Noise level $f$")
        plt.ylabel("MCMC Steps")
        plt.tick_params(axis='both', which='major')
        plt.plot(sigmas,avg_steps_plot,'.-')

        # Overlay distributions at each sigma
        if density:
            for sigma, stps in steps.items():
                if len(stps) > 1 and np.std(stps) > 0: # Ensure there's enough data for KDE
                    kde = gaussian_kde(stps)
                    x_vals = np.linspace(min(stps), max(stps), 1000) # Generate values for KDE
                    y_vals = kde(x_vals)
                    # Normalize the distribution to fit within the plot
                    y_vals = y_vals / max(y_vals) * 0.0075 # Scale factor (adjust as needed)
                    plt.plot(sigma - y_vals, x_vals, color='blue', alpha=0.3)

        plt.grid(True)
        plt.tight_layout()
        image_path = os.path.join(folder_name, "avg_steps_plot.png")
        plt.savefig(image_path)
        plt.show()
    
    if var_steps:
        # Plot the variance of the number of steps.
        plt.title("Standard deviation of the number of steps to find the true alignment (or a better one)")
        plt.xlabel("Noise level $f$")
        plt.ylabel("MCMC Steps")
        plt.tick_params(axis='both', which='major')
        plt.plot(sigmas,var_steps_plot,'.-')
        plt.grid(True)
        plt.tight_layout()
        image_path = os.path.join(folder_name, "std_steps_plot.png")
        plt.savefig(image_path)
        plt.show()
    
    if avg_H_gaps:
        # Plot the average difference between the best energy found and the ground true energy.
        plt.title(r"Average of $H_{min} - H_{true}$")
        plt.xlabel("Noise level $f$",fontsize=24)
        plt.ylabel("Energy (H)",fontsize=24)
        plt.tick_params(axis='both', which='major')
        plt.plot(sigmas,avg_H_gaps_plot,'.-')
        plt.grid(True)
        plt.tight_layout()
        image_path = os.path.join(folder_name, "avg_H_gaps_plot.png")
        plt.savefig(image_path)
        plt.show()
    
    if var_H_gaps:
        # Plot the variance difference between the best energy found and the ground true energy.
        plt.title(r"Standard deviation of $H_{min} - H_{true}$")
        plt.xlabel("Noise level $f$")
        plt.ylabel("Energy (H)")
        plt.tick_params(axis='both', which='major')
        plt.plot(sigmas,var_H_gaps_plot,'.-')
        plt.grid(True)
        plt.tight_layout()
        image_path = os.path.join(folder_name, "std_H_gaps_plot.png")
        plt.savefig(image_path)  # Save as a PNG file.
        plt.show()

#     print(last_prob_plot)
#     print(min_prob_plot)
#     print(align_plot)
#     print(avg_steps_plot)
#     print(var_steps_plot)
#     print(avg_H_gaps_plot)
#     print(var_H_gaps_plot)

## Auxiliary Functions (for other purposes)

In [16]:
def plot_graphs(graph_list,L=None,alignment=None):
    """
    Shows the graphs that are present in 'graph_list' and optionally the blueprint L.
    
    Inputs:
        graph_list: list of nx.DiGraph()
            List of graphs to be plotted (typically here will go the noisy graph observations).
            If only one graph is specified, it must be in between brackets
            
            Example:
                >>> plot_graphs(G)
                        this will return an error, so instead do the following:
                >>> plot_graphs([G])
        
        L: nx.DiGraph(), optional (default: L=None)
            Graph indicating the blueprint L. It nothing is imputed, then it isn't plotted.
            This is useful after MCMC has run, to compare the observations with the obtained blueprint L.
            
        alignment: np.matrix, optional (default: alignment=None)
            A matrix where each row corresponds to a particular observation, and each column represents the alignment 
            of nodes across observations.
            
            Example:
                [[0,2,1]
                 [0,1,2]
                 [2,1,0]
                 [1,0,2]]
            This would be an alignment with 4 observations containing three nodes each. In this case,
            the nodes 0, 0, 2 and 1 from observations 0, 1, 2 and 3 (respectively) are mapped to
            the same node in the blueprint; and so on for the rest of the nodes (columns).
    
    Returns:
        None
            This function doesn't return anything, but rather plots the observations and the blueprint if specified.
    
    """
        
    n_graphs = len(graph_list)
    n_last_graphs = n_graphs % 3
    n_groups = 1+(n_graphs-n_last_graphs)//3
    n_nodes = graph_list[0].number_of_nodes()
    
    if L is not None:
        graph_list = graph_list+[L]
    
    pos_initial = nx.spring_layout(graph_list[0]) # The position of the plot of the first observation (A^0) is stored, so that all observations are plotted in the same way (not changing the distribution of the vertices in the plot).
    only_positions = [positions for _,positions in pos_initial.items()]
    
    # Create an 'n_groups'x3 grid
    fig, axs = plt.subplots(n_groups, 3, figsize=(12, n_groups*4))
    
    # Flatten axs in case there's only one row, so axs[i//3, i%3] works
    if n_groups == 1:
        axs = axs.reshape(1, -1)
        
    pos_list = []
    if alignment is not None:
        n_iter = n_graphs
        if L is not None:
            n_iter += 1
        for i in range(n_iter):
            pos = {alignment[i][j]: only_positions[j] for j in range(n_nodes)}
            pos_list.append(pos)
    else:
        # If no alignment is provided, use the same position for all graphs
        pos_list = [pos_initial] * len(graph_list)
            
            
    # Plot each graph on a separate subplot
    for i, graph in enumerate(graph_list):
        if i != n_graphs: # The position of the blueprint in 'graph_list'!
            axs[i//3, i%3].set_title(f"Observation {i}")
        else:
            axs[i//3, i%3].set_title(f"Blueprint L")
        
        nx.draw(graph, pos=pos_list[i], ax=axs[i//3, i%3], with_labels=True, font_weight='bold', node_color='skyblue', node_size=500, arrows=True)

    # Turn off unused subplots, if any
    for j in range(n_graphs, n_groups*3):
        axs[j//3, j%3].axis("off")

    # Adjust layout to avoid overlap
    plt.tight_layout()

    # Show the plot
    plt.show()

In [17]:
def H_trues_per_sigma(iters):
    start = time.time()
    H_trues = []
    for sigma in np.linspace(0,1,41):
        sigma = round(sigma,5)
        H_tr = []
        for _ in range(iters):
            G = nx.gnp_random_graph(100, 0.1, directed=True)
            obs = create_noisy_copies(graph=G, K=2, sigma=sigma)

            alignment = np.tile(np.arange(100), (2,1)) # True alignment
            adj_matrix = np.array([nx.adjacency_matrix(ob, nodelist=alignment[k]).toarray() for k, ob in enumerate(obs)], dtype=np.int32)
            E = np.sum(adj_matrix, axis=0)

            h = energy_config_H(observations=obs,alignment=alignment,L=create_L(E,2))/100
            H_tr.append(h)

        H_trues.append((sigma,np.mean(H_tr)))

    end = time.time()
    print(f"Time: {end-start} seconds.\n")
    
    return H_trues

In [18]:
def perturb_alignment(true_alignment, n):
    """
    Generate a perturbed initial alignment by cycling n nodes in the alignment.
    
    Parameters:
        true_alignment (ndarray): Shape (n_obs, n_nodes), the true alignment.
        n (int): Number of nodes to cycle.
    
    Returns:
        ndarray: A perturbed alignment.
    """
    perturbed_alignment = true_alignment.copy()  # Avoid modifying original
    
    n_obs, n_nodes = true_alignment.shape
    
    # Select n random unique indices
    selected_indices = np.random.choice(n_nodes, size=n, replace=False)
    
    # Cycle the selected nodes
    # Create a new list where the last element goes to the first place
    cycled_indices = np.roll(selected_indices, shift=1)
    
    # Swap the nodes
    for i in range(n):
        perturbed_alignment[1, selected_indices[i]] = true_alignment[1, cycled_indices[i]]

    print(f'Number of swapped nodes: {np.sum(true_alignment != perturbed_alignment)}')

    return perturbed_alignment

In [19]:
def plot_energies_equi(file_path,folder_name=None):
    with open(file_path, mode='r', newline='') as file:
        reader = csv.reader(file)
        
        # Loop through the rows in the CSV file
        for row in reader:
            if row[0].startswith("Steps required"):
                final_steps = int(row[1])
                
            elif row[0] == "H_min and H_true":
                H_true = float(row[2])
                
            elif row[0] == "Replica 1":
                T = float(row[1])
                H = [float(h) for h in row[2:]]
                
                lower_H = float(row[2])
                min_H = [lower_H]
                
                for h in row[3:]:
                    if float(h) < lower_H:
                        lower_H = float(h)
                    min_H.append(lower_H)
        
        
        final_steps = len(H) if final_steps == -1 else final_steps
        # Plot energies.
        plt.figure(figsize=(25,10))
        plt.title(f"Energy per Node over MCMC Steps in Equilibrium from Replica at T=1",fontsize=25)
        plt.xlabel("MCMC step",fontsize=18)
        plt.ylabel("Energy per node",fontsize=18)
        
#         plt.axhline(y=H_true, color='blue', linestyle='--', label="Ground true energy per node")
        
        # Plot the true energy as a reference.
        plt.plot(range(1000,11*final_steps//12), H[1000:11*final_steps//12], color='pink', label='Energies in equilibrium')
        plt.plot(range(1000,11*final_steps//12), min_H[1000:11*final_steps//12], color='black', label='Lowest energy at the moment')
        plt.legend(fontsize=18,loc='best')
        
        if folder_name:
            image_path = os.path.join(folder_name, "energies_equi" + file_path.split("results")[1].split(".csv")[0] + ".png")
        else:
            image_path = "energies_equi" + file_path.split("results")[1].split(".csv")[0] + ".png"
            
        plt.savefig(image_path)  # Save as a PNG file

In [38]:
def energies_hist(n_graphs, sigma, n_nodes=100, edge_dens=0.1, n_obs=2, max_mcmc_steps=500000, create_graph=True, obs=None, verbose=True):
    """    
    """
    
    global_data = []
    mus = []
    sigmas = []
    plt.figure(figsize=(20,10))
    
    for iter_graph in range(n_graphs):
        if create_graph:
            G = nx.gnp_random_graph(n_nodes, edge_dens, directed=True)
            obs = create_noisy_copies(graph=G, K=n_obs, sigma=sigma)
    
        # Initial parameters.
    #     n_obs = len(obs)
    #     n_nodes = obs[0].number_of_nodes()
        list_of_nodes = np.array(obs[0].nodes())

        # Ground true energy per node.
        alignment = np.tile(np.arange(n_nodes), (n_obs,1))
        adj_matrices = np.array([nx.adjacency_matrix(ob, nodelist=alignment[k]).toarray() for k, ob in enumerate(obs)], dtype=np.int32)
        E = np.sum(adj_matrices, axis=0)
        H_true = energy_config_H(obs,alignment,create_L(E,n_obs))/n_nodes

        # Initial alignment (set to random in this case!)
        alignment = np.array([np.random.permutation(n_nodes) for _ in range(n_obs)])
        
        # Save the adjacency matrices of all the observations.                    
        adj_matrices = np.array([nx.adjacency_matrix(ob, nodelist=alignment[k]).toarray() for k, ob in enumerate(obs)], dtype=np.int32)

        # Now we will initialise the parameters according to the initial alignment.
        E = np.sum(adj_matrices, axis=0)
        O_00,O_01,O_10,O_11,n_0,n_1 = 0,0,0,0,0,0
        all_positions = [(a,b) for a in range(n_nodes) for b in range(n_nodes) if a!=b]

        for i,j in all_positions:
            if E[i,j] > 0.5*n_obs:
                n_1 += 1
                O_11 += E[i,j]
                O_10 += n_obs - E[i,j]

            else:
                n_0 += 1
                O_01 += E[i,j]
                O_00 += n_obs - E[i,j]


        # Calculate the initial energy.
        alpha_p,beta_p,alpha_q,beta_q = 0.2,10,0.5,5
        params = np.array([O_00,O_01,O_10,O_11,n_0,n_1,n_obs,alpha_p,beta_p,alpha_q,beta_q], dtype=np.float64)
        H_initial = energy_H(params=params) / n_nodes

        energies = [H_initial]
        
        if H_initial < 0:
            raise ValueError("Energy can't be negative!")

        
        # MCMC algorithm with parallel tempering starts.
        for i in range(max_mcmc_steps-1):

            # Select a random observation and pair of nodes.
            random_obs_idx = np.random.randint(0,n_obs)
            x,y = np.random.choice(n_nodes, size=2, replace=False)
            node_1, node_2 = alignment[random_obs_idx,x], alignment[random_obs_idx,y]

            # Perform a new alignment, swapping the selected nodes, and calculate its new parameters.
            O_00, O_01, O_10, O_11, n_0, n_1, E, adj_matrix = \
            swap_and_update_vars(
                alignment=alignment, 
                adj_matrices_repl=adj_matrices, 
                E_old=E, 
                O_00_old=O_00, 
                O_01_old=O_01, 
                O_10_old=O_10, 
                O_11_old=O_11, 
                n_1_old=n_1, 
                list_of_nodes=list_of_nodes,
                random_obs_idx=random_obs_idx,
                x=x,
                y=y
            )

            alignment[random_obs_idx,x],alignment[random_obs_idx,y] = node_2,node_1
            adj_matrices[random_obs_idx] = adj_matrix

            # Compute the new energy.
            params = np.array([O_00,O_01,O_10,O_11,n_0,n_1,n_obs,alpha_p,beta_p,alpha_q,beta_q], dtype=np.float64)
            H = energy_H(params=params) / n_nodes

            if H < 0:
                raise ValueError("Energy can't be negative!")

            # Save the energy per node.
            energies.append(H)

            if verbose and round((i+1)/max_mcmc_steps * 100,6) % 5 == 0:
                print("Progress --->", round((i+1)/max_mcmc_steps * 100,2),"%")


        m, s = norm.fit(energies)
        mus.append(m)
        sigmas.append(s)
        global_data.append(energies)
        
        if verbose:
            # Format and style
            sns.set_style("whitegrid")
            sns.set_context("talk")
            
            # KDE Plot from Data
            sns.kdeplot(energies, color='dodgerblue', fill=True, linewidth=2.5, label='Empirical KDE')
            
            # Adjusted Gaussian
            x = np.linspace(min(energies), max(energies), 1000)
            fitted_pdf = norm.pdf(x, m, s)
            plt.plot(x, fitted_pdf, color='limegreen', linestyle='--', linewidth=2.5, label='Fitted Gaussian')
             
            # Title and axis
            plt.title("Energy per Node Density Distribution", fontsize=32, pad=20)
            plt.xlabel("Energy per node $(H)$", fontsize=25, labelpad=15)
            plt.ylabel("Density", fontsize=25, labelpad=15)
            # plt.yscale('log')
            
            # Spacing, grid and legend
            plt.xticks(fontsize=16)
            plt.yticks(fontsize=16)
            plt.grid(True, which="both", linestyle="--")
            plt.legend(fontsize=22, frameon=True, loc='best')
            plt.tight_layout()
                        
            # sns.histplot(energies, bins=n_bins, stat="density", color='dodgerblue', kde=True, alpha=0.6, edgecolor='black', label='Empirical')
            # plt.hist(energies, bins=250, color='blue', alpha=0.7)  
        #     sns.kdeplot(energies, fill=True, color='blue') 
            # plt.legend(loc='best',fontsize=20)

            # Second plot — zoomed-in tail
            plt.figure(figsize=(10, 6))
            sns.set_style("whitegrid")
            sns.set_context("talk")
        
            # KDE with xlim
            sns.kdeplot(energies, color='dodgerblue', fill=True, linewidth=2.5, label='Empirical KDE')
        
            # Recompute x and fitted_pdf restricted to [46, 48]
            x_zoom = np.linspace(54, 59, 1000)
            fitted_pdf_zoom = norm.pdf(x_zoom, m, s)
            plt.plot(x_zoom, fitted_pdf_zoom, color='limegreen', linestyle='--', linewidth=2.5, label='Fitted Gaussian')
        
            # Axes limits for zoom
            plt.xlim(54, 59)
            plt.yscale('log')

            # Optional: Set a reasonable y-limit if the density is small in that region
            # You can also inspect `fitted_pdf_zoom` or use plt.ylim(0, 0.02)
            # plt.ylim(0, 0.02)
        
            plt.title("Zoom on Tail: Energy per Node Distribution", fontsize=24, pad=20)
            plt.xlabel("Energy per node $(H)$", fontsize=18, labelpad=15)
            plt.ylabel("Density", fontsize=18, labelpad=15)
        
            plt.xticks(fontsize=16)
            plt.yticks(fontsize=16)
            plt.grid(True, which="both", linestyle="--")
            plt.legend(fontsize=16, frameon=True, loc='best')
            plt.tight_layout()
            plt.show()
    

        progress = int(100*(iter_graph+1)/n_graphs)
        if progress % 5 == 0:
            print(f"{progress}%")
    
    global_mu, global_sigma = norm.fit(global_data)
    avg_mu, avg_sigma = np.mean(mus), np.mean(sigmas)
    
    H_min = min(energies)
    H_max = max(energies)
    plt.savefig("GaussianEnergy.png", dpi=300, bbox_inches='tight')
    plt.show()
    
    return global_mu, global_sigma, avg_mu, avg_sigma, H_min, H_max, H_true

In [21]:
def plot_gaussian_true_H(graph,sigma_range,itr,max_mcmc_steps=500000,K=2):
    """
    Plots the evolution of energy distribution parameters and ground truth energy
    against different noise levels.

    Inputs:
        graph: nx.DiGraph()
            The base graph for generating noisy copies.
            
        sigma_range: array
            The range of sigma values (proportion of swapped edges) to be tested.
            
        max_mcmc_steps: int
            Maximum number of sampled  for each noise level.
            
        K: int
            Number of observations to generate per sigma level.
        
    Returns:
        None
    """
    
    mus_avg,sigmas_avg = [],[]
    H_mins_avg, H_maxs_avg = [], []  # avg
#     H_mins, H_maxs = [], [] # global
    H_trues_avg = []
    H_trues_mins, H_trues_maxs = [], []
    
    for sgm in sigma_range:
        print(f"\nSigma: {round(sgm,2)}")
        mus, sigmas, H_trues, H_mins, H_maxs = [], [], [], [], [] # last 2: avg
#         H_mn = float('inf') # global
#         H_mx = 0 # global
        H_true_min = float('inf')
        H_true_max = 0
        for i in range(itr):
            print(f"Iteration: {i+1}")    
            observations = create_noisy_copies(graph=graph,K=K,sigma=sgm)
            _, _, mu, sigma, H_min, H_max, H_true = energies_hist(n_graphs=1, sigma=sgm, max_mcmc_steps=max_mcmc_steps, create_graph=False, obs=observations, verbose=False)

            mus.append(mu)
            sigmas.append(sigma)
            H_mins.append(H_min) # avg
#             H_mn = min(H_mn,H_min) # global
            H_maxs.append(H_max) # avg
#             H_mx = max(H_mx,H_max) # global
            H_trues.append(H_true)

            H_true_min = min(H_true,H_true_min)
            H_true_max = max(H_true,H_true_max)
                
        
        mus_avg.append(np.mean(mus))
        sigmas_avg.append(np.mean(sigmas))
        H_mins_avg.append(np.mean(H_mins))
#         H_mins.append(H_mn) # global
        H_maxs_avg.append(np.mean(H_maxs)) # avg
#         H_maxs.append(H_mx) # global
        H_trues_avg.append(np.mean(H_trues)) # avg
        
        H_trues_mins.append(H_true_min)
        H_trues_maxs.append(H_true_max)
        
        
    # Convert lists to numpy arrays for easy calculations
    mus_avg = np.array(mus_avg)
    sigmas_avg = np.array(sigmas_avg)
    H_mins_avg = np.array(H_mins_avg) # avg
#     H_mins = np.array(H_mins) # global
    H_maxs_avg = np.array(H_maxs_avg) # avg
#     H_maxs = np.array(H_maxs) # global
    H_trues_avg = np.array(H_trues_avg)
    H_trues_mins = np.array(H_trues_mins)
    H_trues_maxs = np.array(H_trues_maxs)
    
    # Plot all the information.
    plt.figure(figsize=(25,10))
    plt.title("Sampled Alignments vs Ground True Energy",fontsize=28)
    plt.xlabel("Proportion of changed edges (σ)",fontsize=20)
    plt.ylabel("Energy per Node",fontsize=20)

    # Plot μ
    plt.plot(sigma_range, mus_avg, label="μ", color="blue")

    # Plot μ +- σ
    upper_bound_mu = mus_avg + sigmas_avg
    lower_bound_mu = mus_avg - sigmas_avg
    
    plt.plot(sigma_range, upper_bound_mu, color='blue', linestyle='--', label="μ + σ")
    plt.plot(sigma_range, lower_bound_mu, color='blue', linestyle='--', label="μ - σ")
    plt.fill_between(sigma_range, lower_bound_mu, upper_bound_mu, color='blue', alpha=0.2)

    # Plot H_min, H_max
    plt.plot(sigma_range, H_mins_avg, color='green', linestyle='--', label=r"$H_{min}$") # avg
#     plt.plot(sigma_range, H_mins, color='green', linestyle='--', label=r"$H_{min}$") # global
    plt.plot(sigma_range, H_maxs_avg, color='green', linestyle='--', label=r"$H_{max}$") # avg
#     plt.plot(sigma_range, H_maxs, color='green', linestyle='--', label=r"$H_{max}$") # global
    plt.fill_between(sigma_range, H_mins_avg, H_maxs_avg, color='green', alpha=0.2) # avg
#     plt.fill_between(sigma_range, H_mins, H_maxs, color='green', alpha=0.2) # global

    # Plot H_true
    plt.plot(sigma_range, H_trues_avg, label=r"$H_{true}$", color='red', linestyle='-')

    # Plot H_true_max, H_true_min
    plt.plot(sigma_range, H_trues_mins, color='red', linestyle='--', label=r"$H_{true,min}$")
    plt.plot(sigma_range, H_trues_maxs, color='red', linestyle='--', label=r"$H_{true,max}$")
    plt.fill_between(sigma_range, H_trues_mins, H_trues_maxs, color='red', alpha=0.2)
    
    plt.legend(loc='best',fontsize=20)
    plt.show()

## Deprecated Functions

In [22]:
def select_random_nodes(alignment, positions, seeds=None):
    """
    Randomly selects two nodes from the same observation in the alignment matrix for swapping. The nodes are selected 
    from different groups (group-labeled nodes, or nodes with no information) according to the positions 
    dictionary. Seed nodes are excluded from swapping.

    Inputs:
        alignment: np.matrix
            A matrix where each row corresponds to a particular observation, and each column represents the alignment 
            of nodes across observations. Nodes are grouped first by seeds, then by group-labeled nodes, and finally
            by nodes with no additional information (which are aligned based on their degree).

        positions: dict
            A dictionary that tracks the column positions for each node category (seeds, group-labeled nodes, and nodes 
            without additional information). The keys are the category names, and the values are the column ranges 
            corresponding to those categories in the 'alignment' matrix.

            Example:
                positions = {
                    "seeds": range(0, 2), 
                    "group_label_0": range(2, 4), 
                    "group_label_1": range(4, 7), 
                    "no info": range(7, 10)
                }

            This indicates that the first two columns correspond to seeds, columns 2 and 3 to group label 0,
            columns 4 to 6 to group label 1, and columns 7 to 9 to nodes without additional information.

        seeds: dict, optional (default: seeds=None)
            A dictionary containing the seed nodes for each observation. Seed nodes are pre-aligned and cannot be 
            selected for swapping. The keys are observation indices, and the values are lists of seed node indices.

            Example:
                seeds = {0: [0,1], 1: [0,1], 2: [0,1], 3: [0,1], 4: [0,1]}

            In this example, nodes 0 and 1 in all observations are seeds and will not be selected for swapping.

    Returns:
        node_1, node_2: nx.DiGraph() node
            The two nodes that have been selected to be swapped at a certain MCMC step.

        random_obs_idx: int
            The index of the randomly selected observation from which the nodes are chosen. This identifies which row 
            of the alignment matrix is used.

        random_idx_node_1, random_idx_node_2: int
            The column indices in the alignment matrix of the two randomly selected nodes. This identifies which columns
            of the alignment matrix are used.

    Raises:
        ValueError:
            If a seed node is selected for swapping. The function already handles seeds and doesn't swap them; however
            this exception is still raised as a way to check that everything works just fine.

    """
    
    # Number of seeds.
    n_seeds = len(seeds[0]) if seeds else 0
    
    # Total number of observations and nodes.
    n_obs,n_nodes = alignment.shape[0],alignment.shape[1]
    
    # Randomly select an observation.
    random_obs_idx = random.randint(0,n_obs-1)

    # Randomly select a node index between n_seeds and n_nodes.
    random_idx_node_1 = random.randint(n_seeds, n_nodes-1)
    
    # Determine which group the node belongs to by checking the positions.
    for node_group, position_range in positions.items():
        if random_idx_node_1 in position_range:
            
            # SEED:
            if node_group == "seeds":
                raise ValueError("A seed has been selected for a swap. However, this shouldn't happen, as seed indices have been removed.")
            
            # GROUP_LABEL:
            elif node_group.startswith("group_label"):
                # Randomly select another index from the same group.
                random_idx_node_2 = random.choice([i for i in position_range if i != random_idx_node_1])
            
            # NO INFO:
            else:
                # Randomly select another index from the nodes without information.
                random_idx_node_2 = random.choice([i for i in position_range if i != random_idx_node_1])
                
            # Select the two selected nodes for swapping.
            node_1 = alignment[random_obs_idx,random_idx_node_1]
            node_2 = alignment[random_obs_idx,random_idx_node_2]
#             print(f"Positions {random_idx_node_1},{random_idx_node_2}")
#             print(f"Observation {random_obs_idx}: nodes {node_1},{node_2}")
    
            return node_1,node_2,random_obs_idx,random_idx_node_1,random_idx_node_2

In [23]:
def most_common_alignment(alignments):
    """
    """
    
    # Stack matrices into a 3D array (n_matrices, n_rows, n_cols)
    alignments_stack = np.array(alignments)
    
    # Calculate the mode along the first axis (across matrices) with keepdims=True
    mode_alignments, _ = mode(alignments_stack, axis=0, keepdims=True)
    
    # 'mode_alignments' returns a 3D array, squeeze it to 2D
    return np.squeeze(mode_alignments)